In [23]:
#import lightgbm as lgb
import joblib
from lightgbm import LGBMClassifier
import numpy as np
from sklearn.model_selection import StratifiedKFold, train_test_split, cross_val_score
from sklearn.metrics import make_scorer
import optuna
import optuna.integration.lightgbm as lgb
from optuna.integration import LightGBMPruningCallback
from lightgbm import LGBMClassifier
#from optuna.integration._lightgbm_tuner.sklearn import LGBMClassifier
from utils import g_score

In [29]:
X_train = np.load('X_train.npy',allow_pickle=True)
y_train = np.load('y_train.npy')
feature_names = list(np.load('feature_names.npy'))
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

In [30]:
X_train = X_train[:20000]
y_train = y_train[:20000]

In [15]:
def objective(trial, X_train, y_train, cv):
    param_grid = {
        "n_estimators": trial.suggest_categorical("n_estimators", [10000]),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "num_leaves": trial.suggest_int("num_leaves", 20, 3000, step=20),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 200, 10000, step=100),
        "lambda_l1": trial.suggest_int("lambda_l1", 0, 100, step=5),
        "lambda_l2": trial.suggest_int("lambda_l2", 0, 100, step=5),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 0, 15),
        "bagging_fraction": trial.suggest_float(
            "bagging_fraction", 0.2, 0.95, step=0.1
        ),
        "bagging_freq": trial.suggest_categorical("bagging_freq", [1]),
        "feature_fraction": trial.suggest_float(
            "feature_fraction", 0.2, 0.95, step=0.1
        ),
    }
    
    model = LGBMClassifier(**param_grid)
    return cross_val_score(model, X_train, y_train, scoring=g_score, cv=cv, n_jobs=-1).mean()

In [31]:
optuna.logging.set_verbosity(optuna.logging.INFO)
sampler = optuna.samplers.TPESampler(seed=1)
study = optuna.create_study(direction="maximize", study_name="LGBM_ht", sampler=sampler)
joblib.dump(study, "LGBM_ht.pkl")
func = lambda trial: objective(trial, X_train, y_train, cv)
try:
    study.optimize(func, n_jobs= -1, timeout= 100, show_progress_bar=True)  #n_trials=20
    joblib.dump(study, "LGBM_ht.pkl")
except Exception as error:
    joblib.dump(study, "LGBM_ht.pkl")
    print(error)



[I 2023-02-10 06:35:50,261] A new study created in memory with name: LGBM_ht
/home/user/.local/share/virtualenvs/dmeyf-kP6Zbi_a/lib/python3.8/site-packages/optuna/study/_optimize.py:57: UserWarning: The timeout-based progress bar is not supported with n_jobs != 1.
  warnings.warn("The timeout-based progress bar is not supported with n_jobs != 1.")
/home/user/.local/share/virtualenvs/dmeyf-kP6Zbi_a/lib/python3.8/site-packages/optuna/distributions.py:701: UserWarning: The distribution is specified by [0.2, 0.95] and step=0.1, but the range is not divisible by `step`. It will be replaced by [0.2, 0.9].
  warnings.warn(
/home/user/.local/share/virtualenvs/dmeyf-kP6Zbi_a/lib/python3.8/site-packages/optuna/distributions.py:701: UserWarning: The distribution is specified by [0.2, 0.95] and step=0.1, but the range is not divisible by `step`. It will be replaced by [0.2, 0.9].
  warnings.warn(
/home/user/.local/share/virtualenvs/dmeyf-kP6Zbi_a/lib/python3.8/site-packages/optuna/distributions.py

[LightGBM] [Warning] lambda_l1 is set=0, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0
[LightGBM] [Warning] bagging_fraction is set=0.5, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5
[LightGBM] [Warning] min_gain_to_split is set=3.8058627535561356, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=3.8058627535561356
[LightGBM] [Warning] lambda_l2 is set=45, reg_lambda=0.0 will be ignored. Current value: lambda_l2=45
[LightGBM] [Warning] feature_fraction is set=0.2, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2
[LightGBM] [Warning] min_data_in_leaf is set=8800, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=8800
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l1 is set=0, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0
[LightGBM] [Warning] bagging_fraction is set=0.5, subsample=1.0 will be

[I 2023-02-10 06:36:19,570] Trial 0 finished with value: 17904000.0 and parameters: {'n_estimators': 10000, 'learning_rate': 0.17837287633030227, 'num_leaves': 140, 'max_depth': 10, 'min_data_in_leaf': 8800, 'lambda_l1': 0, 'lambda_l2': 45, 'min_gain_to_split': 3.8058627535561356, 'bagging_fraction': 0.5, 'bagging_freq': 1, 'feature_fraction': 0.2}. Best is trial 0 with value: 17904000.0.
/home/user/.local/share/virtualenvs/dmeyf-kP6Zbi_a/lib/python3.8/site-packages/optuna/distributions.py:701: UserWarning: The distribution is specified by [0.2, 0.95] and step=0.1, but the range is not divisible by `step`. It will be replaced by [0.2, 0.9].
  warnings.warn(


[LightGBM] [Warning] lambda_l1 is set=20, reg_alpha=0.0 will be ignored. Current value: lambda_l1=20
[LightGBM] [Warning] bagging_fraction is set=0.30000000000000004, subsample=1.0 will be ignored. Current value: bagging_fraction=0.30000000000000004
[LightGBM] [Warning] min_gain_to_split is set=13.952728375181495, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=13.952728375181495
[LightGBM] [Warning] lambda_l2 is set=35, reg_lambda=0.0 will be ignored. Current value: lambda_l2=35
[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] min_data_in_leaf is set=4000, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=4000
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l1 is set=20, reg_alpha=0.0 will be ignored. Current value: lambda_l1=20
[LightGBM] [Warning] bagging_fracti

[I 2023-02-10 06:37:26,259] Trial 3 finished with value: 17904000.0 and parameters: {'n_estimators': 10000, 'learning_rate': 0.1955233250224563, 'num_leaves': 1560, 'max_depth': 5, 'min_data_in_leaf': 4000, 'lambda_l1': 20, 'lambda_l2': 35, 'min_gain_to_split': 13.952728375181495, 'bagging_fraction': 0.30000000000000004, 'bagging_freq': 1, 'feature_fraction': 0.7}. Best is trial 0 with value: 17904000.0.
/home/user/.local/share/virtualenvs/dmeyf-kP6Zbi_a/lib/python3.8/site-packages/optuna/distributions.py:701: UserWarning: The distribution is specified by [0.2, 0.95] and step=0.1, but the range is not divisible by `step`. It will be replaced by [0.2, 0.9].
  warnings.warn(


...

[I 2023-02-10 06:41:13,216] Trial 4 finished with value: 17904000.0 and parameters: {'n_estimators': 10000, 'learning_rate': 0.05134445766466694, 'num_leaves': 200, 'max_depth': 11, 'min_data_in_leaf': 8900, 'lambda_l1': 100, 'lambda_l2': 20, 'min_gain_to_split': 11.969011090852904, 'bagging_fraction': 0.6000000000000001, 'bagging_freq': 1, 'feature_fraction': 0.5}. Best is trial 1 with value: 18407200.0.


[LightGBM] [Warning] lambda_l1 is set=5, reg_alpha=0.0 will be ignored. Current value: lambda_l1=5
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] min_gain_to_split is set=8.380538283610717, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=8.380538283610717
[LightGBM] [Warning] lambda_l2 is set=30, reg_lambda=0.0 will be ignored. Current value: lambda_l2=30
[LightGBM] [Warning] feature_fraction is set=0.4, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4
[LightGBM] [Warning] min_data_in_leaf is set=2200, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=2200
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


[I 2023-02-10 06:42:05,330] Trial 2 finished with value: 18256800.0 and parameters: {'n_estimators': 10000, 'learning_rate': 0.061485924907781216, 'num_leaves': 2760, 'max_depth': 6, 'min_data_in_leaf': 2300, 'lambda_l1': 65, 'lambda_l2': 50, 'min_gain_to_split': 12.12310600597895, 'bagging_fraction': 0.9, 'bagging_freq': 1, 'feature_fraction': 0.30000000000000004}. Best is trial 1 with value: 18407200.0.


[LightGBM] [Warning] lambda_l1 is set=5, reg_alpha=0.0 will be ignored. Current value: lambda_l1=5
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] min_gain_to_split is set=8.380538283610717, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=8.380538283610717
[LightGBM] [Warning] lambda_l2 is set=30, reg_lambda=0.0 will be ignored. Current value: lambda_l2=30
[LightGBM] [Warning] feature_fraction is set=0.4, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4
[LightGBM] [Warning] min_data_in_leaf is set=2200, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=2200
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l1 is set=5, reg_alpha=0.0 will be ignored. Current value: lambda_l1=5
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be i

[I 2023-02-10 06:44:38,339] Trial 5 finished with value: 18866800.0 and parameters: {'n_estimators': 10000, 'learning_rate': 0.0818879444738594, 'num_leaves': 80, 'max_depth': 10, 'min_data_in_leaf': 2200, 'lambda_l1': 5, 'lambda_l2': 30, 'min_gain_to_split': 8.380538283610717, 'bagging_fraction': 0.8, 'bagging_freq': 1, 'feature_fraction': 0.4}. Best is trial 5 with value: 18866800.0.


In [32]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_bagging_fraction,params_bagging_freq,params_feature_fraction,params_lambda_l1,params_lambda_l2,params_learning_rate,params_max_depth,params_min_data_in_leaf,params_min_gain_to_split,params_n_estimators,params_num_leaves,state
0,0,17904000.0,2023-02-10 06:35:50.277806,2023-02-10 06:36:19.569576,0 days 00:00:29.291770,0.5,1,0.2,0,45,0.178373,10,8800,3.805863,10000,140,COMPLETE
1,1,18407200.0,2023-02-10 06:35:50.559298,2023-02-10 06:38:56.862807,0 days 00:03:06.303509,0.7,1,0.8,45,60,0.105616,11,4700,10.168326,10000,2600,COMPLETE
2,2,18256800.0,2023-02-10 06:35:50.566691,2023-02-10 06:42:05.329282,0 days 00:06:14.762591,0.9,1,0.3,65,50,0.061486,6,2300,12.123106,10000,2760,COMPLETE
3,3,17904000.0,2023-02-10 06:35:50.569035,2023-02-10 06:37:26.258477,0 days 00:01:35.689442,0.3,1,0.7,20,35,0.195523,5,4000,13.952728,10000,1560,COMPLETE
4,4,17904000.0,2023-02-10 06:36:19.578848,2023-02-10 06:41:13.215669,0 days 00:04:53.636821,0.6,1,0.5,100,20,0.051344,11,8900,11.969011,10000,200,COMPLETE
5,5,18866800.0,2023-02-10 06:37:26.268246,2023-02-10 06:44:38.338480,0 days 00:07:12.070234,0.8,1,0.4,5,30,0.081888,10,2200,8.380538,10000,80,COMPLETE


In [26]:
print(f"\tBest value (ganancia): {study.best_value:.5f}")
print(f"\tBest params:")

for key, value in study.best_params.items():
    print(f"\t\t{key}: {value}")

	Best value (ganancia): 832000.00000
	Best params:
		n_estimators: 10000
		learning_rate: 0.2594876362814767
		num_leaves: 2140
		max_depth: 9
		min_data_in_leaf: 700
		lambda_l1: 50
		lambda_l2: 85
		min_gain_to_split: 0.18124776275421972
		bagging_fraction: 0.30000000000000004
		bagging_freq: 1
		feature_fraction: 0.8
